<a href="https://colab.research.google.com/github/QuantAnalyticsTorch/quant_analytics_torch/blob/main/examples/RandomPerformance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
#@title Upgrade to TensorFlow nightly
!pip install --upgrade tf-nightly

     |████████████████████████████████| 398.7MB 44kB/s 
     |████████████████████████████████| 471kB 51.1MB/s 
     |████████████████████████████████| 3.8MB 49.2MB/s 
     |████████████████████████████████| 1.3MB 43.5MB/s 
     |████████████████████████████████| 11.9MB 146kB/s 
     |████████████████████████████████| 14.5MB 210kB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: grpcio 1.33.2
    Uninstalling grpcio-1.33.2:
      Successfully uninstalled grpcio-1.33.2
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18

In [1]:
import tensorflow as tf

In [2]:
from scipy.special import erf
import numpy as np
import time

In [3]:
x = tf.math.sobol_sample(10,10)

In [5]:
import numpy as np

In [6]:
class BrownianBridge():
    def __init__(self, number_time_steps):
        super(BrownianBridge, self).__init__()

        self.number_time_steps = number_time_steps

        self.left_index = np.zeros(number_time_steps, dtype=int)
        self.right_index = np.zeros(number_time_steps, dtype=int)
        self.bridge_index = np.zeros(number_time_steps, dtype=int)
        self.left_weight = np.zeros(number_time_steps)
        self.right_weight = np.zeros(number_time_steps)
        self.std_dev = np.zeros(number_time_steps)

        self._map = np.zeros(number_time_steps, dtype=int)

        self._map[-1] = 1
        self.bridge_index[0] = number_time_steps - 1
        self.std_dev[0] = tf.math.sqrt(1.0 * number_time_steps)
        self.left_weight[0] = 0
        self.right_weight[0] = 0

        j=0
        for i in range(1,number_time_steps):
            while self._map[j] == True:
                j = j + 1
            k = j
            while self._map[k] == False:
                k = k + 1
            l = j+((k-1-j)>>1)
            self._map[l]=i
            self.bridge_index[i]=l
            self.left_index[i]=j
            self.right_index[i]=k
            self.left_weight[i]=(k-l)/(k+1-j)
            self.right_weight[i]=(1+l-j)/(k+1-j)
            self.std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
            j=k+1
            if j>=number_time_steps:
                j=0

    @tf.function(jit_compile=True)
    def buildPath(self, z, number):
        path = tf.unstack(tf.zeros(shape=(self.number_time_steps,number)))
        path[-1] = self.std_dev[0]*z[0];
        for i in np.arange(1,self.number_time_steps):
            j = self.left_index[i]
            k = self.right_index[i]
            l = self.bridge_index[i]
            #if j > 0:
            #    path[l] = self.left_weight[i] * path[j-1] + self.right_weight[i] * path[k] + self.std_dev[i] * z[i]
            #else:
            path[l] = self.right_weight[i] * path[k] + self.std_dev[i] * z[i]

        return path

In [7]:
import tensorflow as tf

In [8]:
import numpy as np
import math

In [9]:
number_time_steps = 256
number = 3

left_index = np.zeros(number_time_steps, dtype=int)
right_index = np.zeros(number_time_steps, dtype=int)
bridge_index = np.zeros(number_time_steps, dtype=int)
left_weight = np.zeros(number_time_steps)
right_weight = np.zeros(number_time_steps)
std_dev = np.zeros(number_time_steps)

_map = np.zeros(number_time_steps, dtype=int)

_map[-1] = 1
bridge_index[0] = number_time_steps - 1
std_dev[0] = math.sqrt(1.0 * number_time_steps)
left_weight[0] = 0
right_weight[0] = 0

j=0
for i in range(1,number_time_steps):
  while _map[j] == True:
    j = j + 1
  k = j
  while _map[k] == False:
    k = k + 1
l = j+((k-1-j)>>1)
_map[l]=i
bridge_index[i]=l
left_index[i]=j
right_index[i]=k
left_weight[i]=(k-l)/(k+1-j)
right_weight[i]=(1+l-j)/(k+1-j)
std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
j=k+1
if j>=number_time_steps:
  j=0

bridge_index_t = tf.constant(bridge_index)
right_index_t = tf.constant(right_index)
left_index_t = tf.constant(left_index)

In [18]:
#@tf.function(jit_compile=True)
def buildPath(z, number, number_time_steps, left_index_t, right_index_t, bridge_index_t, left_weight, right_weight, std_dev):
  path = tf.unstack(tf.zeros(shape=(number_time_steps,number),dtype=tf.dtypes.float64))
  path[-1] = std_dev[0]*z[0];
  j = 0
  k = 0
  l = 0
  i = 0
  for i in range(1,number_time_steps):
#    print("i")
#    print(i)
    j = left_index[i]
    k = right_index[i]
    l = bridge_index[i]
#    print("l")
#    print(l)
#    print("k")
#    print(k)
#    print("right_weight")
#    print(right_weight[i])
    path[l] = right_weight[i] * path[k] + std_dev[i] * z[i]

  return path

In [11]:
x = tf.math.sobol_sample(256,5,dtype=tf.dtypes.float64)
z = tf.transpose(tf.math.erf(x))

In [12]:
path = buildPath(z, 5, 256, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)

In [13]:
print(tf.autograph.to_code(buildPath.python_function))

def tf__buildPath(z, number, number_time_steps, left_index_t, right_index_t, bridge_index_t, left_weight, right_weight, std_dev):
    with ag__.FunctionScope('buildPath', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        path = ag__.converted_call(ag__.ld(tf).unstack, (ag__.converted_call(ag__.ld(tf).zeros, (), dict(shape=(ag__.ld(number_time_steps), ag__.ld(number)), dtype=ag__.ld(tf).dtypes.float64), fscope),), None, fscope)
        ag__.ld(path)[(- 1)] = (ag__.ld(std_dev)[0] * ag__.ld(z)[0])
        j = 0
        k = 0
        l = 0
        i = 0

        def get_state():
            return ()

        def set_state(block_vars):
            pass

        def loop_body(itr):
            i = itr
            j = ag__.ld(left_index)[ag__.ld(i)]
            k = ag__.ld(right_index)[ag__.ld(i)]
            l = ag__.ld(bridge

In [14]:
bridge = BrownianBridge(2)

In [19]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  256#@param {type:"integer"}
number =  100000#@param {type:"integer"}

# First run (includes graph optimization time)
time_start = time.time()
with tf.device("/cpu:0"):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
    z = tf.transpose(tf.math.erf(x))
    w = buildPath(z, number, 128, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)
time_end = time.time()
time_price_cpu = time_end - time_start
print("First time on a CPU: ", time_price_cpu)

# Second run (excludes graph optimization time)
time_start = time.time()
with tf.device("/cpu:0"):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
    z = tf.transpose(tf.math.erf(x))
    w = buildPath(z, number, 128, left_index, right_index, bridge_index, left_weight, right_weight, std_dev)
    print(w[0])
time_end = time.time()

time_price_cpu = time_end - time_start
print("Second time on a CPU: ", time_price_cpu)

First time on a CPU:  0.5446352958679199
tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(100000,), dtype=float64)
Second time on a CPU:  0.5020267963409424


In [20]:
import torch

In [21]:
@torch.jit.script
def foo(x, number: int):
  print(number)
  z = 0.
  for i in range(0,number):
    tmp = x[i]
    z = z + tmp

# See the compiled graph as Python code
print(foo.code)

def foo(x: Tensor,
    number: int) -> None:
  print(number)
  z = 0.
  for _0 in range(torch.__range_length(0, number, 1)):
    i = torch.__derive_index(_0, 0, 1)
    tmp = torch.select(x, 0, i)
    z0 = torch.add(tmp, z, 1)
    z = annotate(float, z0)
  return None



In [22]:
# Call the function using the TorchScript interpreter
x = torch.zeros(11)

foo(x, 10)

10


In [23]:
@torch.jit.script
def buildPath(z, number: int, number_time_steps: int, left_index, right_index, bridge_index, left_weight, right_weight, std_dev):
  path = torch.zeros(size=(number_time_steps,number))
  path[-1] = std_dev[0]*z[0]
  j = 0
  k = 0
  l = 0
  i = 0
  for i in range(1,number_time_steps):
#    print("i")
#    print(i)
    j = left_index[i]
    k = right_index[i]
    l = bridge_index[i]
#    print("l")
#    print(l)
#    print("k")
#    print(k)
#    print("right_weight")
#    print(right_weight[i])
    path[l] = right_weight[i] * path[k] + std_dev[i] * z[i]

  return path

In [24]:
print(buildPath.code)

def buildPath(z: Tensor,
    number: int,
    number_time_steps: int,
    left_index: Tensor,
    right_index: Tensor,
    bridge_index: Tensor,
    left_weight: Tensor,
    right_weight: Tensor,
    std_dev: Tensor) -> Tensor:
  path = torch.zeros([number_time_steps, number], dtype=None, layout=None, device=None, pin_memory=None)
  _0 = torch.mul(torch.select(std_dev, 0, 0), torch.select(z, 0, 0))
  _1 = torch.copy_(torch.select(path, 0, -1), _0, False)
  _2 = torch.__range_length(1, number_time_steps, 1)
  for _3 in range(_2):
    i = torch.__derive_index(_3, 1, 1)
    k = torch.select(right_index, 0, i)
    _4 = annotate(int, k)
    l = torch.select(bridge_index, 0, i)
    _5 = annotate(int, l)
    _6 = torch.mul(torch.select(right_weight, 0, i), torch.select(path, 0, _4))
    _7 = torch.mul(torch.select(std_dev, 0, i), torch.select(z, 0, i))
    _8 = torch.add(_6, _7, alpha=1)
    _9 = torch.copy_(torch.select(path, 0, _5), _8, False)
  return path



In [25]:
x = torch.randn(256,2)

left_index_t = torch.tensor(left_index)
right_index_t = torch.tensor(right_index)
bridge_index_t = torch.tensor(bridge_index)
left_weight_t = torch.tensor(left_weight)
right_weight_t = torch.tensor(right_weight)
std_dev_t = torch.tensor(std_dev)

path = buildPath(x, 2, 256, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)

In [26]:
import time

In [27]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  256#@param {type:"integer"}
number =  100000#@param {type:"integer"}

# First run (includes graph optimization time)
time_start = time.time()
x = torch.randn(dim, number)
path = buildPath(x, number, dim, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)
time_end = time.time()
time_price_cpu = time_end - time_start
print("First time on a CPU: ", time_price_cpu)

# Second run (excludes graph optimization time)
time_start = time.time()
x = torch.randn(dim, number)
path = buildPath(x, number, dim, left_index_t, right_index_t, bridge_index_t, left_weight_t, right_weight_t, std_dev_t)
time_end = time.time()

time_price_cpu = time_end - time_start
print("Second time on a CPU: ", time_price_cpu)

First time on a CPU:  0.40483903884887695
Second time on a CPU:  0.3911397457122803
